In [1]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

class ImageDataset(Dataset):
    def __init__(self, image_dir, transform = None):
        self.image_dir = image_dir
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform
    def __len__(self):
        return len(self.image_files)
    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])
image_folder_path = "./images"
dataset = ImageDataset(image_folder_path, transform = transform)
dataloader = DataLoader(dataset, batch_size =16, shuffle = True)


        

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

class SuperResolutionModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Define your layers here
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 3, kernel_size=3, padding=1)
    
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return x

model = SuperResolutionModel()
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [3]:
from PIL import Image
import torch
from diffusers import StableDiffusionInpaintPipeline
import os

# Load Stable Diffusion Inpainting Pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", torch_dtype=torch.float16
).to(device)

# Function to generate mask for zoom-out gaps
def create_mask(image, scale=0.5):
    original_width, original_height = image.size
    new_width = int(original_width * scale)
    new_height = int(original_height * scale)

    # Create blank mask (black background)
    mask = Image.new("L", (original_width, original_height), 0)

    # Define white region for inpainting (zoomed-out area)
    paste_x = (original_width - new_width) // 2
    paste_y = (original_height - new_height) // 2
    white_box = (paste_x, paste_y, paste_x + new_width, paste_y + new_height)
    mask.paste(255, white_box)

    return mask

# Function to perform inpainting
def inpaint_image(image_path, output_path, prompt="Fill missing areas"):
    # Load image
    image = Image.open(image_path).convert("RGB")

    # Generate mask for zoom-out gaps
    mask = create_mask(image)

    # Perform inpainting using Stable Diffusion
    result = pipe(prompt=prompt, image=image, mask_image=mask).images[0]

    # Save output image
    result.save(output_path)

# Directory paths
image_folder_path = "./images"
output_folder_path = "./output_images"
os.makedirs(output_folder_path, exist_ok=True)

# Process each image in the folder
for filename in os.listdir(image_folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(image_folder_path, filename)
        output_path = os.path.join(output_folder_path, filename)
        inpaint_image(img_path, output_path)

print(f"Processed images saved to {output_folder_path}")


ModuleNotFoundError: No module named 'diffusers'

In [ ]:
import pkg_resources
pkg_resources.require('diffusers')
